# Đánh index văn bản

In [ ]:
import docx
import json

doc = docx.Document('67_2014_QH13_259729.docx')

document_json = {
    "document" : []
}
# put a placeholder for later indexing
for para in doc.paragraphs:
    document_json["document"].append({
        "index" : "0.0.0",
        "text" : para.text
	})
    
# save the json file
with open('67_2014_QH13_259729_only_dieu.json', 'w', encoding="utf-8") as json_file:
	json.dump(document_json, json_file, indent=4, ensure_ascii=False)

In [ ]:
import re
import json

with open('67_2014_QH13_259729_only_dieu.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	document_data = data["document"]

document_data_len = len(document_data)
data_for_print = []

for i in range(1, document_data_len):
	previous_element_index_list = document_data[i-1]["index"].split(".")
	previous_element_section_number = int(previous_element_index_list[0])
	previous_element_subsection_number = int(previous_element_index_list[1])
	element = document_data[i]
	if element["text"].startswith("Chương"):
		# remove that element from the list
		continue
	# elif element["text"] starts with "Mục " and a number
	elif re.match(r"^Mục \d+", element["text"]):
		# remove that element from the list
		continue
	elif element["text"].startswith("Điều"):
		# section number is the first word right after "Điều", remove the dot at the end
		section_number = element["text"].split()[1][:-1]
		# update the index
		index_list = element["index"].split(".")
		index_list[0] = section_number
		element["index"] = ".".join(index_list)
		data_for_print.append({
			"index" : element["index"],
			"text" : element["text"]
		})
	elif re.match(r"^\d+\.", element["text"]):
		# subsection number is the first word, remove the dot at the end
		subsection_number = element["text"].split(".")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[1] = subsection_number
		index_list[0] = str(previous_element_section_number)
		element["index"] = ".".join(index_list)
		data_for_print.append({
			"index" : element["index"],
			"text" : element["text"]
		})
	# else if element["text"] starts with a character and a ) then it is a point, update the index
	elif re.match(r"^[a-zđê]\)", element["text"], re.IGNORECASE):
		# point number is the first character, remove the bracket at the end
		point_number = element["text"].split(")")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[2] = point_number
		index_list[1] = str(previous_element_subsection_number)
		index_list[0] = str(previous_element_section_number)
		element["index"] = ".".join(index_list)
		data_for_print.append({
			"index" : element["index"],
			"text" : element["text"]
		})
	else:
		index_list = element["index"].split(".")
		index_list[0] = str(previous_element_section_number)
		element["index"] = ".".join(index_list)
		data_for_print.append({
			"index" : element["index"],
			"text" : element["text"]
		})

# save document_data to json file
with open('67_2014_QH13_259729_only_dieu_index.json', 'w', encoding="utf-8") as json_file:
	json.dump({
		"document" : data_for_print
	}, json_file, indent=4, ensure_ascii=False)

### Convert sang dictionary (để truy vấn)

In [ ]:
import json

with open('67_2014_QH13_259729_only_dieu_index.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	document_data = data["document"]

document_index = {}
document_data_len = len(document_data)

for i in range(document_data_len):
	element = document_data[i]
	if element["index"] not in document_index:
		document_index.update({element["index"] : element["text"]})
	else:
		document_index[element["index"]] += " " + element["text"]

# save document_index to json file
with open('67_2014_QH13_259729_only_dieu_index_another_way.json', 'w', encoding="utf-8") as json_file:
	json.dump(document_index, json_file, indent=4, ensure_ascii=False)

# Tìm kiếm các tên cơ quan có xuất hiện trong văn bản

In [ ]:
state_agencies = {
    "quoc hoi",
    "uy ban thuong vu quoc hoi",
    "doan chu tich uy ban trung uong mat tran to quoc viet nam",
    "chinh phu",
    "chu tich nuoc",
    "thu tuong chinh phu",
    "hoi dong tham phan toa an nhan dan toi cao",
    "toa an nhan dan toi cao",
    "vien kiem sat nhan dan toi cao",
    "tong kiem toan nha nuoc",
    "bo quoc phong",
    "bo cong an",
    "bo ngoai giao",
    "bo tu phap",
    "bo tai chinh",
    "bo cong thuong",
    "bo lao dong thuong binh va xa hoi",
    "bo giao thong van tai",
    "bo xay dung",
    "bo thong tin va truyen thong",
    "bo giao duc va dao tao",
    "bo nong nghiep va phat trien nong thon",
    "bo ke hoach va dau tu",
    "bo noi vu",
    "bo y te",
    "bo khoa hoc va cong nghe",
    "bo van hoa the thao va du lich",
    "bo tai nguyen va moi truong",
    "van phong chinh phu",
    "than tra chinh phu",
    "ngan hang nha nuoc",
    "uy ban dan toc",
    "hoi dong nhan dan",
    "uy ban nhan dan",
}

In [ ]:
state_agencies_vn = [
    "quốc hội",
    "ủy ban thường vụ quốc hội",
    "đoàn chủ tịch ủy ban trung ương mặt trận tổ quốc việt nam",
    "chính phủ",
    "chủ tịch nước",
    "thủ tướng chính phủ",
    "hội đồng thẩm phán tòa án nhân dân tối cao",
    "tòa án nhân dân tối cao",
    "viện kiểm sát nhân dân tối cao",
    "tổng kiểm toán nhà nước",
    "bộ, cơ quan ngang bộ",
    "bộ quốc phòng",
    "bộ công an",
    "bộ ngoại giao",
    "bộ tư pháp",
    "bộ tài chính",
    "bộ công thương",
    "bộ lao động thương binh và xã hội",
    "bộ giao thông vận tải",
    "bộ xây dựng",
    "bộ thông tin và truyền thông",
    "bộ giáo dục và đào tạo",
    "bộ nông nghiệp và phát triển nông thôn",
    "bộ kế hoạch và đầu tư",
    "bộ nội vụ",
    "bộ y tế",
    "bộ khoa học và công nghệ",
    "bộ văn hóa thể thao và du lịch",
    "bộ tài nguyên và môi trường",
    "văn phòng chính phủ",
    "thanh tra chính phủ",
    "ngân hàng nhà nước",
    "ủy ban dân tộc",
    "hội đồng nhân dân",
    "ủy ban nhân dân",
]

In [ ]:
import json

with open('67_2014_QH13_259729_only_dieu_index.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	document_data = data["document"]

jurisdict = []

for element in document_data:
	for agency in state_agencies_vn:
		if agency in element["text"].lower():
			jurisdict.append({
				"index": element["index"],
				"text": element["text"],
				"agency": agency,
				"jurisdiction": "temporarily empty",
				"authority": "temporarily empty"
			})
			# print(element["index"], " ", element["text"])

with open('67_2014_QH13_259729_only_dieu_jurisdict.json', 'w', encoding="utf-8") as json_file:
	json.dump({"jurisdict": jurisdict}, json_file, ensure_ascii=False)
	

# Bóc tách thẩm quyền

Cell dưới dùng để tìm đến "điểm a khoản 1 Điều này" được nhắc đến ở đoạn sau.

Dự án đầu tư quy định tại điểm a khoản 1 Điều này thực hiện tại khu công nghiệp, khu chế xuất, khu công nghệ cao, khu kinh tế phù hợp với quy hoạch đã được cấp có thẩm quyền phê duyệt không phải trình Ủy ban nhân dân cấp tỉnh quyết định chủ trương đầu tư.

Out put là 1 dictionary có keys:

	- index: index của câu gốc
	- text: full text
	- agency: cơ quan có liên quan
	- jurisdiction: thẩm quyền của họ
	- authority: từ mang ý nghĩa phân quyền
	- reference: nội dung được refer đến (ví dụ điểm a khoản 1 Điều này)
	- higher_level_reference: index có level cao hơn gần đoạn text nhất

{
	"index": "16.3.0",
	"text": "3. Bãi bỏ việc trợ cấp, miễn, giảm học phí cho các đối tượng quy định tại Điểm đ, Khoản 3, Điều 33 của Nghị định số 75/2006/NĐ-CP ngày 02 tháng 8 năm 2006 của Chính phủ quy định chi tiết và hướng dẫn thi hành một số điều của Luật Giáo dục và chế độ ưu đãi đối với học sinh, sinh viên các bộ môn nghệ thuật truyền thống và đặc thù trong các trường văn hóa - nghệ thuật quy định tại Khoản 2, Điều 2, Quyết định số 82/2005/QĐ-TTg ngày 18 tháng 4 năm 2005 của Thủ tướng Chính phủ về chế độ ưu đãi đối với học sinh, sinh viên các bộ môn nghệ thuật truyền thống và đặc thù trong các trường văn hóa - nghệ thuật.",
	"agency": "thủ tướng chính phủ",
	"jurisdiction": " và hướng dẫn thi hành một số điều của Luật Giáo dục và chế độ ưu đãi đối với học sinh, sinh viên các bộ môn nghệ thuật truyền thống và đặc thù trong các trường văn hóa - nghệ thuật quy định tại Khoản 2, Điều 2, Quyết định số 82/2005/QĐ-TTg ngày 18 tháng 4 năm 2005 của Thủ tướng Chính phủ về chế độ ưu đãi đối với học sinh, sinh viên các bộ môn nghệ thuật truyền thống và đặc thù trong các trường văn hóa - nghệ thuật.",
	"authority": "quy định chi tiết",
	"reference": "Điều 2. Đối tượng áp dụng",
	"higher_level_reference": "Điều 16. Hiệu lực thi hành Nghị định"
}

### Lưu ý, còn nhiều lỗi, cần tinh chỉnh thêm

In [ ]:
import json
import re

with open('67_2014_QH13_259729_only_dieu_jurisdict.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	main_data = data["jurisdict"]
with open('67_2014_QH13_259729_only_dieu_index_another_way.json', 'r', encoding="utf-8") as json_file_2:
	ref_data = json.load(json_file_2)

allow_appear_before_keyword = ["được quyết định bởi"]
not_allow_appear_before_keyword = ["không được quyết định bởi"]
allow_appear_after_keyword = ["quy định", "quyết định", "có thẩm quyền", "quy định chi tiết"]
not_allow_appear_after_keyword = ["không được ban hành"]

process_data = []
agency_and_jurisdiction = {
	"allow" : {},
	"not_allow" : {}
}

before_keyword_list = allow_appear_before_keyword + not_allow_appear_before_keyword
after_keyword_list = allow_appear_after_keyword + not_allow_appear_after_keyword

def extract_numbers(text, start, end):
    # Slicing the text from start to end index
    sliced_text = text[start:end+1]
    # Using regular expression to find all numbers in the sliced text
    numbers = re.findall(r'\b\d+\b', sliced_text)
    # Converting found numbers from strings to integers
    return [number for number in numbers]

def extract_characters(text, start_index, end_index):
    # Extract the substring based on the provided indices
    substring = text[start_index:end_index + 1]

    # Use regular expression to find all standalone characters
    # \b is a word boundary, \w is any word character, and {1} specifies exactly one occurrence of a word character
    import re
    pattern = r'\b\w{1}\b'
    alone_characters = re.findall(pattern, substring)

    return alone_characters

def extract_reference(index, text):
	dieu_list = []
	khoan_list = []
	diem_list = []
	index_list = index.split(".")
	text_lower = text.lower()
	if "điều này" in text_lower:
        # Get first appear of "Điều này"
		dieu_nay_index = text_lower.find("điều này")
		if "khoản" in text_lower and text_lower.find("khoản") < dieu_nay_index:
			khoan_index = text_lower.find("khoản")
			if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
				# Get first appear Điểm index
				diem_index = text_lower.find("điểm")
				diem_list = extract_characters(text_lower, diem_index, khoan_index)
			khoan_list = extract_numbers(text_lower, khoan_index, dieu_nay_index)
			dieu_list.append(index_list[0])
		else:
			dieu_list.append(index_list[0])
	else:
		if "điều" in text_lower:
            # Get first appear of "Điều"
			dieu_index = text_lower.find("điều")
			if "khoản" in text_lower and text_lower.find("khoản") < dieu_index:
                # Get first appear Khoản index
				khoan_index = text_lower.find("khoản")
				if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_characters(text_lower, diem_index, khoan_index)
                # for character in text from khoan_index to dieu_index
				khoan_list = extract_numbers(text_lower, khoan_index, dieu_index)
				dieu_list = extract_numbers(text_lower, dieu_index, dieu_index+10)
			else:
				dieu_list = extract_numbers(text_lower, dieu_index, len(text_lower)-1)
				if len(index_list) > 10:
					dieu_list = dieu_list[:10]
		else:
			if "khoản" in text_lower:
                # Get first appear Khoản index
				khoan_index = text_lower.find("khoản")
				if "điểm" in text_lower and text_lower.find("điểm") < khoan_index:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_characters(text_lower, diem_index, khoan_index)
                # for character in text from khoan_index to end of text
				khoan_list = extract_numbers(text_lower, khoan_index, len(text_lower)-1)
				if len(index_list) > 10:
					khoan_list = khoan_list[:10]
				dieu_list.append(index_list[0])
			else:
				if "điểm" in text_lower:
					# Get first appear Điểm index
					diem_index = text_lower.find("điểm")
					diem_list = extract_numbers(text_lower, diem_index, len(text_lower)-1)
					if len(index_list) > 10:
						diem_list = diem_list[:10]
					khoan_list.append(index_list[1])
					dieu_list.append(index_list[0])
	return dieu_list, khoan_list, diem_list

def get_reference(index, text):
	dieu_list, khoan_list, diem_list = extract_reference(index, text)
	if len(dieu_list) == 1:
		if len(khoan_list) > 1:
			for khoan in khoan_list:
				ref_index = f"{dieu_list[0]}.{khoan}.0"
				return ref_data[ref_index] if ref_index in ref_data else ""
		else:
			for diem in diem_list:
				ref_index = f"{dieu_list[0]}.{khoan_list[0]}.{diem}"
				return ref_data[ref_index] if ref_index in ref_data else ""
	else:
		for dieu in dieu_list:
			ref_index = f"{dieu}.0.0"
			return ref_data[ref_index] if ref_index in ref_data else ""
		
def get_higher_level_index(index):
	index_list = index.split(".")
	if index_list[2] != "0":
		return ref_data[f"{index_list[0]}.{index_list[1]}.0"]
	elif index_list[2] == "0" and index_list[1] != "0":
		return ref_data[f"{index_list[0]}.0.0"]
	
for element in main_data:
	for before_keyword in before_keyword_list:
		# if before_keyword in element["text"] and before_keyword appear before element["agency"], extract the text before before_keyword from element["text"]
		if before_keyword in element["text"] and element["text"].find(before_keyword) < element["text"].find(element["agency"]):
			index = element["text"].find(before_keyword)
			process_data.append({
				"index": element["index"], 
				"text": element["text"],
				"agency": element["agency"],
				"jurisdiction": element["text"][:index],
				"authority": before_keyword,
				"reference": get_reference(element["index"], element["text"][:index]),
				"higher_level_reference": get_higher_level_index(element["index"])
				})
			jurisdiction_text = element["text"][:index]
			reference_text = get_reference(element["index"], element["text"][:index])
			higher_level_reference_text = get_higher_level_index(element["index"])
			get_reference(element["index"], element["text"][:index])
			# if element["agency"] is an existed key in agency_and_jurisdiction, append the jurisdiction to the existed key
			if before_keyword in allow_appear_before_keyword:
				if element["agency"] in agency_and_jurisdiction["allow"]:
					agency_and_jurisdiction["allow"][element["agency"]].append(
						jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["allow"][element["agency"]] = [element["text"][:index]]
			else:
				if element["agency"] in agency_and_jurisdiction["not_allow"]:
					agency_and_jurisdiction["not_allow"][element["agency"]].append(
						jurisdiction_text + " " + 
						(reference_text if reference_text is not None else "") + " " + 
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["not_allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "")
					]

	for after_keyword in after_keyword_list:
		# if after_keyword in element["text"] and after_keyword appear after element["agency"], extract the text after after_keyword from element["text"]
		if after_keyword in element["text"] and element["text"].find(after_keyword) > element["text"].find(element["agency"]):
			index = element["text"].find(after_keyword)
			process_data.append({
				"index": element["index"], 
				"text": element["text"],
				"agency": element["agency"],
				"jurisdiction": element["text"][index + len(after_keyword):],
				"authority": after_keyword,
				"reference": get_reference(element["index"], element["text"][index + len(after_keyword):]),
				"higher_level_reference": get_higher_level_index(element["index"])
			})
			jurisdiction_text = element["text"][index + len(after_keyword):]
			reference_text = get_reference(element["index"], element["text"][index + len(after_keyword):])
			higher_level_reference_text = get_higher_level_index(element["index"])
			# get_reference(element["index"], element["text"][index + len(after_keyword):])
			if after_keyword in allow_appear_after_keyword:
				if element["agency"] in agency_and_jurisdiction["allow"]:
					agency_and_jurisdiction["allow"][element["agency"]].append(
						jurisdiction_text + " " +
						(reference_text if reference_text is not None else "") + " " +
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "") + " "
					]
			else:
				if element["agency"] in agency_and_jurisdiction["not_allow"]:
					agency_and_jurisdiction["not_allow"][element["agency"]].append(
						(jurisdiction_text if jurisdiction_text is not None else "") + " " +
						(reference_text if reference_text is not None else "") + " " +
						(higher_level_reference_text if higher_level_reference_text is not None else "")
					)
				else:
					agency_and_jurisdiction["not_allow"][element["agency"]] = [
						jurisdiction_text + " " +
						(reference_text if reference_text else "") + " " +
						(higher_level_reference_text if higher_level_reference_text else "")
					]

# save the data to json file
with open('67_2014_QH13_259729_only_dieu_jurisdict_process.json', 'w', encoding="utf-8") as json_file_3:
	json.dump({"jurisdiction" : process_data}, json_file_3, ensure_ascii=False, indent=4)

with open('67_2014_QH13_259729_only_dieu_jurisdict_agency_jurisdiction.json', 'w', encoding="utf-8") as json_file_4:
	json.dump(agency_and_jurisdiction, json_file_4, ensure_ascii=False, indent=4)

# Phụ lục: Index theo 4 trường - Chương điều khoản điểm

In [ ]:
import docx
import json

doc = docx.Document('67_2014_QH13_259729.docx')

document_json = {
    "document" : []
}

for para in doc.paragraphs:
    document_json["document"].append({
        "index" : "0.0.0.0",
        "text" : para.text
	})
    
# save the json file
with open('67_2014_QH13_259729.json', 'w', encoding="utf-8") as json_file:
	json.dump(document_json, json_file, indent=4, ensure_ascii=False)

In [ ]:
import re
import json

with open('67_2014_QH13_259729.json', 'r', encoding="utf-8") as json_file:
	data = json.load(json_file)
	document_data = data["document"]

def roman_to_int(s):
	rom_val = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
	int_val = 0
	for i in range(len(s)):
		if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
			int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
		else:
			int_val += rom_val[s[i]]
	return int_val

document_data_len = len(document_data)

for i in range(1, document_data_len):
	previous_element_index_list = document_data[i-1]["index"].split(".")
	previous_element_chapter_number = int(previous_element_index_list[0])
	previous_element_section_number = int(previous_element_index_list[1])
	previous_element_subsection_number = int(previous_element_index_list[2])
	element = document_data[i]
	# if element["text"] starts with "Chương" then it is a chapter, update the index
	if element["text"].startswith("Chương"):
		# chapter number is the first word right after "Chương"
		chapter_number = element["text"].split()[1]
		# right now the chapter number is roman numeral, convert it to arabic numeral
		chapter_number = roman_to_int(chapter_number)
		# update the index
		index_list = element["index"].split(".")
		index_list[0] = str(chapter_number)
		element["index"] = ".".join(index_list)
	elif element["text"].startswith("Điều"):
		# section number is the first word right after "Điều", remove the dot at the end
		section_number = element["text"].split()[1][:-1]
		# update the index
		index_list = element["index"].split(".")
		index_list[0] = str(previous_element_chapter_number)
		index_list[1] = section_number
		element["index"] = ".".join(index_list)
	elif re.match(r"^\d+\.", element["text"]):
		# subsection number is the first word, remove the dot at the end
		subsection_number = element["text"].split(".")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[2] = subsection_number
		index_list[1] = str(previous_element_section_number)
		index_list[0] = str(previous_element_chapter_number)
		element["index"] = ".".join(index_list)
	# else if element["text"] starts with a character and a ) then it is a point, update the index
	elif re.match(r"^[a-zđê]\)", element["text"], re.IGNORECASE):
		# point number is the first character, remove the bracket at the end
		point_number = element["text"].split(")")[0]
		# update the index
		index_list = element["index"].split(".")
		index_list[3] = point_number
		index_list[2] = str(previous_element_subsection_number)
		index_list[1] = str(previous_element_section_number)
		index_list[0] = str(previous_element_chapter_number)
		element["index"] = ".".join(index_list)
	else:
		index_list = element["index"].split(".")
		index_list[0] = str(previous_element_chapter_number)
		index_list[1] = str(previous_element_section_number)
		element["index"] = ".".join(index_list)

# save document_data to json file
with open('67_2014_QH13_259729_index.json', 'w', encoding="utf-8") as json_file:
	json.dump(data, json_file, indent=4, ensure_ascii=False)	